Щербаков Павел, гр. 3821Б1ПМоп2

**Для подробного описания данных см. другой файл**

1. Вся предобработка данных с того файла

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
raw_data = pd.read_csv("data.txt",sep=";")
raw_data = raw_data.drop("duration",axis = 1)
raw_data = raw_data[raw_data['age'] < 70.0]
raw_data = raw_data[raw_data['campaign'] < 7]
raw_data = raw_data[raw_data['previous'] <= 2.0]
cols = raw_data.columns
from sklearn.impute import SimpleImputer
imp_num = SimpleImputer(missing_values='unknown', strategy="most_frequent")
imp_num.fit(raw_data)
raw_data = pd.DataFrame(imp_num.transform(raw_data))
raw_data.columns = cols
for col_name in ['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome', 'y']:
    raw_data[col_name] = raw_data[col_name].astype('category')
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='if_binary', sparse_output=False)
X = raw_data.drop(['y'],axis = 1)
enc.fit(X[['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome']])
dummies = pd.DataFrame(enc.transform(X[['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome']]),
                       columns=enc.get_feature_names_out(), index=X.index)
X = pd.concat((X, dummies), axis=1).drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome'], axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns, index=X.index)
y = raw_data['y'].map({'yes': 1.0, 'no': 0.0})

Без оверсемплинга нормальные результаты по кластеризации не получались

In [4]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

Делим на обучающую и тестовую выборку

In [6]:
from sklearn.model_selection import train_test_split
X_res_train, X_res_test, y_res_train, y_res_test = train_test_split(
    X_res, y_res, test_size=0.3, random_state=42)

2. Кластеризация

In [8]:
from sklearn.cluster import DBSCAN,KMeans,Birch
from sklearn import metrics

В качестве объективной метрики выберем Adjusted Rand Score

2. 1. DBSCAN - метод, основанный на плотности

Используем optuna для максимизации ARI по гиперпараметрам

In [26]:
import optuna

In [32]:
def objective(trial):
    eps = trial.suggest_float('eps', 1.0, 5.0)
    min_samples = trial.suggest_int('min_samples',100,200)
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(X_res_train)
    return metrics.adjusted_rand_score(y_res_train, clustering.labels_)

In [34]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
study.best_params

[I 2024-12-15 14:17:27,533] A new study created in memory with name: no-name-e09245db-cc87-43c4-9399-72359fbcb9e6
[I 2024-12-15 14:17:31,374] Trial 0 finished with value: 0.0 and parameters: {'eps': 2.6242136334269874, 'min_samples': 182}. Best is trial 0 with value: 0.0.
[I 2024-12-15 14:17:35,250] Trial 1 finished with value: 0.0 and parameters: {'eps': 1.3048990307806432, 'min_samples': 118}. Best is trial 0 with value: 0.0.
[I 2024-12-15 14:17:42,428] Trial 2 finished with value: 0.001162785259191219 and parameters: {'eps': 3.3531129958874417, 'min_samples': 186}. Best is trial 2 with value: 0.001162785259191219.
[I 2024-12-15 14:17:46,048] Trial 3 finished with value: 4.830791767802875e-07 and parameters: {'eps': 1.9441647769697967, 'min_samples': 120}. Best is trial 2 with value: 0.001162785259191219.
[I 2024-12-15 14:17:50,304] Trial 4 finished with value: 0.052331960626185905 and parameters: {'eps': 4.937297641228676, 'min_samples': 146}. Best is trial 4 with value: 0.052331960

{'eps': 4.376347944067499, 'min_samples': 176}

In [36]:
clustering = DBSCAN(eps=4.376347944067499, min_samples=176).fit(X_res_test)
metrics.adjusted_rand_score(y_res_test, clustering.labels_)

0.021078873985542014

2. 2. KMeans - метод, основанный на близости

In [47]:
def objective(trial):
    num_clusters = trial.suggest_int('num_clusters',1,10)
    clustering = KMeans(n_clusters = num_clusters).fit(X_res_train)
    return metrics.adjusted_rand_score(y_res_train, clustering.labels_)

In [49]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
study.best_params

[I 2024-12-15 14:33:46,075] A new study created in memory with name: no-name-51ab4b94-fd1a-4b06-8cca-d222601f9a73
[I 2024-12-15 14:33:46,397] Trial 0 finished with value: 0.06423420353645869 and parameters: {'num_clusters': 9}. Best is trial 0 with value: 0.06423420353645869.
[I 2024-12-15 14:33:46,588] Trial 1 finished with value: 0.10774059463549385 and parameters: {'num_clusters': 4}. Best is trial 1 with value: 0.10774059463549385.
[I 2024-12-15 14:33:46,740] Trial 2 finished with value: 0.02775301109408576 and parameters: {'num_clusters': 2}. Best is trial 1 with value: 0.10774059463549385.
[I 2024-12-15 14:33:46,904] Trial 3 finished with value: 0.14536670778479632 and parameters: {'num_clusters': 3}. Best is trial 3 with value: 0.14536670778479632.
[I 2024-12-15 14:33:47,079] Trial 4 finished with value: 0.10297156130024834 and parameters: {'num_clusters': 6}. Best is trial 3 with value: 0.14536670778479632.
[I 2024-12-15 14:33:47,282] Trial 5 finished with value: 0.066521729358

{'num_clusters': 2}

In [51]:
clustering = KMeans(n_clusters = 2).fit(X_res_test)
metrics.adjusted_rand_score(y_res_test, clustering.labels_)

0.002891237043303151

2. 3. BIRCH - метод, основанный на деревьях

In [58]:
def objective(trial):
    num_clusters = trial.suggest_int('num_clusters',1,10)
    clustering = Birch(n_clusters = num_clusters).fit(X_res_train)
    return metrics.adjusted_rand_score(y_res_train, clustering.labels_)

In [60]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
study.best_params

[I 2024-12-15 14:44:10,108] A new study created in memory with name: no-name-918ca5c1-fcea-4ca3-8598-4da69cfe3af2
[I 2024-12-15 14:45:22,219] Trial 0 finished with value: 0.08587742484591322 and parameters: {'num_clusters': 8}. Best is trial 0 with value: 0.08587742484591322.
[I 2024-12-15 14:46:33,284] Trial 1 finished with value: 0.08943037126589587 and parameters: {'num_clusters': 6}. Best is trial 1 with value: 0.08943037126589587.
[I 2024-12-15 14:47:45,970] Trial 2 finished with value: 0.08587742484591322 and parameters: {'num_clusters': 8}. Best is trial 1 with value: 0.08943037126589587.
[I 2024-12-15 14:48:58,922] Trial 3 finished with value: 0.03149364601723845 and parameters: {'num_clusters': 2}. Best is trial 1 with value: 0.08943037126589587.
[I 2024-12-15 14:50:14,956] Trial 4 finished with value: 0.10181138261094957 and parameters: {'num_clusters': 4}. Best is trial 4 with value: 0.10181138261094957.
[I 2024-12-15 14:51:26,664] Trial 5 finished with value: 0.089430371265

{'num_clusters': 3}

In [62]:
clustering = Birch(n_clusters = 3).fit(X_res_test)
metrics.adjusted_rand_score(y_res_test, clustering.labels_)

0.09507147411421792

**Сравнение методов**

In [68]:

table1 = [["DBSCAN", 0.021078873985542014],
         ["KMeans", 0.002891237043303151],
         ["Birch", 0.09507147411421792]]

In [70]:
pd.DataFrame(table1,columns=["Метод","Лучший ARI" ])

,Метод,Лучший ARI
0,DBSCAN,0.021079
1,KMeans,0.002891
2,Birch,0.095071


**Выводы**:
- Кластеризация происходит плохо даже с оверсэмплингом
- Использованные методы кластеризации нужные кластеры не находят
- Лучший результат получился с использованием метода кластеризации Birch (ARI = $0.095071$)